In [1]:
import pandas as pd

df = pd.read_csv('atpbind3d_stats.csv')

df.groupby('model_key').agg({'mcc': ['mean', 'std'], 'micro_auprc': ['mean', 'std']}).sort_values(('micro_auprc', 'mean'), ascending=False)

mcc           micro_auprc          
                                  mean       std        mean       std
model_key                                                             
esm-t33-gearnet-ensemble       0.69206  0.005896     0.71934  0.008358
esm-t33-gearnet-resiboost-r10  0.69302  0.010933     0.71140  0.007343
esm-t33-gearnet-adaboost-r10   0.68558  0.005043     0.70360  0.008797
esm-t33-gearnet                0.67614  0.015672     0.68694  0.006360
esm-t33                        0.66474  0.011330     0.67786  0.018335
bert-gearnet                   0.56634  0.022893     0.57330  0.026640
gearnet                        0.47780  0.014725     0.49974  0.008557
bert                           0.45796  0.028289     0.45350  0.020081

In [1]:
import pandas as pd

df = pd.read_csv('atpbind3d_stats.csv')

df.groupby('model_key').agg({'mcc': ['mean', 'std'], 'micro_auprc': [
    'mean', 'std']}).sort_values(('micro_auprc', 'mean'), ascending=False)

mcc           micro_auprc          
                                  mean       std        mean       std
model_key                                                             
esm-t33-gearnet-resiboost-r10  0.69266  0.007966     0.71218  0.006018
esm-t33-gearnet-adaboost-r10   0.68730  0.009608     0.70158  0.011219
esm-t33                        0.66022  0.018397     0.67628  0.017926

In [2]:
df2 = df[['model_key', 'valid_fold', 'mcc', 'micro_auprc', 'sensitivity', 'precision']]

In [3]:
df2.groupby(['model_key', 'valid_fold']).agg({'mcc': ['mean', 'std'], 'micro_auprc': ['mean', 'std'], 'sensitivity': ['mean', 'std'], 'precision': ['mean', 'std']}).sort_values(('mcc', 'mean'), ascending=False)

mcc     micro_auprc      \
                                            mean std        mean std   
model_key                     valid_fold                               
esm-t33-gearnet-resiboost-r10 3           0.7064 NaN      0.7176 NaN   
esm-t33-gearnet-ensemble      2           0.6966 NaN      0.7312 NaN   
esm-t33-gearnet-resiboost-r10 2           0.6966 NaN      0.7132 NaN   
esm-t33-gearnet-ensemble      1           0.6965 NaN      0.7232 NaN   
esm-t33-gearnet               1           0.6959 NaN      0.6919 NaN   
esm-t33-gearnet-ensemble      3           0.6946 NaN      0.7182 NaN   
esm-t33-gearnet-resiboost-r10 1           0.6931 NaN      0.7179 NaN   
                              4           0.6929 NaN      0.7078 NaN   
esm-t33-gearnet-adaboost-r10  1           0.6905 NaN      0.7106 NaN   
esm-t33-gearnet-ensemble      4           0.6899 NaN      0.7149 NaN   
esm-t33-gearnet-adaboost-r10  2           0.6880 NaN      0.7065 NaN   
                              4           0.6872 NaN      0.6981 NaN   
esm-t33-gearnet               3           0.6853 NaN      0.6840 NaN   
esm-t33-gearnet-adaboost-r10  3           0.6849 NaN      0.7117 NaN   
esm-t33-gearnet-ensemble      0           0.6827 NaN      0.7092 NaN   
esm-t33                       1           0.6814 NaN      0.7014 NaN   
esm-t33-gearnet               2           0.6783 NaN      0.6953 NaN   
esm-t33-gearnet-adaboost-r10  0           0.6773 NaN      0.6911 NaN   
esm-t33-gearnet-resiboost-r10 0           0.6761 NaN      0.7005 NaN   
esm-t33                       2           0.6715 NaN      0.6867 NaN   
esm-t33-gearnet               4           0.6638 NaN      0.6833 NaN   
esm-t33                       3           0.6590 NaN      0.6826 NaN   
esm-t33-gearnet               0           0.6574 NaN      0.6802 NaN   
esm-t33                       4           0.6566 NaN      0.6600 NaN   
                              0           0.6552 NaN      0.6586 NaN   
bert-gearnet                  2           0.5935 NaN      0.6019 NaN   
                              1           0.5887 NaN      0.6029 NaN   
                              4           0.5547 NaN      0.5509 NaN   
                              3           0.5491 NaN      0.5564 NaN   
                              0           0.5457 NaN      0.5544 NaN   
gearnet                       1           0.4967 NaN      0.5109 NaN   
bert                          1           0.4961 NaN      0.4837 NaN   
gearnet                       4           0.4816 NaN      0.4982 NaN   
                              3           0.4787 NaN      0.5060 NaN   
                              0           0.4764 NaN      0.4912 NaN   
bert                          4           0.4682 NaN      0.4431 NaN   
                              2           0.4628 NaN      0.4576 NaN   
gearnet                       2           0.4556 NaN      0.4924 NaN   
bert                          0           0.4414 NaN      0.4536 NaN   
                              3           0.4213 NaN      0.4295 NaN   

                                         sensitivity     precision      
                                                mean std      mean std  
model_key                     valid_fold                                
esm-t33-gearnet-resiboost-r10 3               0.6869 NaN    0.7528 NaN  
esm-t33-gearnet-ensemble      2               0.6350 NaN    0.7911 NaN  
esm-t33-gearnet-resiboost-r10 2               0.6217 NaN    0.8073 NaN  
esm-t33-gearnet-ensemble      1               0.6751 NaN    0.7459 NaN  
esm-t33-gearnet               1               0.6439 NaN    0.7792 NaN  
esm-t33-gearnet-ensemble      3               0.6499 NaN    0.7698 NaN  
esm-t33-gearnet-resiboost-r10 1               0.6617 NaN    0.7534 NaN  
                              4               0.6884 NaN    0.7250 NaN  
esm-t33-gearnet-adaboost-r10  1               0.6617 NaN    0.7483 NaN  
esm-t33-gearnet-ensemble      4               0.6543 NaN    0.7551 NaN  

In [4]:
import pandas as pd
from tabulate import tabulate

def format_mean_std(row, metric):
    mean_val = row[(metric, 'mean')]
    std_val = row[(metric, 'std')]
    return f"{mean_val:.3f} ± {std_val:.3f}"


def get_stat_df(filename, verbose=True):
    # Sample loading of your CSV - this would be replaced with your actual loading
    result_df = pd.read_csv(filename)

    metrics = ['micro_auprc', 'mcc',  'sensitivity', 'precision']
    grouped = result_df.groupby('model_key').agg({
        k: ['mean', 'std'] for k in metrics
    })

    record_counts = result_df.groupby('model_key').size()

    for metric in metrics:
        grouped[f'{metric}_formatted'] = grouped.apply(lambda row: format_mean_std(row, metric), axis=1)


    # Add the record count column to the grouped dataframe
    grouped['record_count'] = record_counts.values

    formatted_df = grouped[[k + '_formatted' for k in metrics] + ['record_count']]
    formatted_df.columns = metrics + ['record_count']
    formatted_df = formatted_df.sort_values(metrics[0], ascending=True)

    formatted_df = formatted_df.reset_index()
    
    if verbose:
        markdown_table = tabulate(formatted_df[['model_key'] + metrics] , headers='keys', tablefmt='pipe', showindex=False)
        print(markdown_table)

    return formatted_df

        
get_stat_df('atpbind3d_stats.csv')

| model_key                     | micro_auprc   | mcc           | sensitivity   | precision     |
|:------------------------------|:--------------|:--------------|:--------------|:--------------|
| bert                          | 0.454 ± 0.020 | 0.458 ± 0.028 | 0.373 ± 0.045 | 0.618 ± 0.101 |
| gearnet                       | 0.500 ± 0.009 | 0.478 ± 0.015 | 0.413 ± 0.047 | 0.603 ± 0.051 |
| bert-gearnet                  | 0.573 ± 0.027 | 0.566 ± 0.023 | 0.493 ± 0.039 | 0.690 ± 0.016 |
| esm-t33                       | 0.678 ± 0.018 | 0.665 ± 0.011 | 0.613 ± 0.024 | 0.751 ± 0.015 |
| esm-t33-gearnet               | 0.687 ± 0.006 | 0.676 ± 0.016 | 0.626 ± 0.021 | 0.760 ± 0.037 |
| esm-t33-gearnet-adaboost-r10  | 0.704 ± 0.009 | 0.686 ± 0.005 | 0.642 ± 0.024 | 0.761 ± 0.021 |
| esm-t33-gearnet-resiboost-r10 | 0.711 ± 0.007 | 0.693 ± 0.011 | 0.667 ± 0.028 | 0.749 ± 0.039 |
| esm-t33-gearnet-ensemble      | 0.719 ± 0.008 | 0.692 ± 0.006 | 0.650 ± 0.016 | 0.765 ± 0.017 |


,model_key,micro_auprc,mcc,sensitivity,precision,record_count
0,bert,0.454 ± 0.020,0.458 ± 0.028,0.373 ± 0.045,0.618 ± 0.101,5
1,gearnet,0.500 ± 0.009,0.478 ± 0.015,0.413 ± 0.047,0.603 ± 0.051,5
2,bert-gearnet,0.573 ± 0.027,0.566 ± 0.023,0.493 ± 0.039,0.690 ± 0.016,5
3,esm-t33,0.678 ± 0.018,0.665 ± 0.011,0.613 ± 0.024,0.751 ± 0.015,5
4,esm-t33-gearnet,0.687 ± 0.006,0.676 ± 0.016,0.626 ± 0.021,0.760 ± 0.037,5
5,esm-t33-gearnet-adaboost-r10,0.704 ± 0.009,0.686 ± 0.005,0.642 ± 0.024,0.761 ± 0.021,5
6,esm-t33-gearnet-resiboost-r10,0.711 ± 0.007,0.693 ± 0.011,0.667 ± 0.028,0.749 ± 0.039,5
7,esm-t33-gearnet-ensemble,0.719 ± 0.008,0.692 ± 0.006,0.650 ± 0.016,0.765 ± 0.017,5
